# CITS4403 - P2P Modelling

This notebook demonstrates a complex P2P model using agent gossip protocols (GNUTELLA-like) and graph-theory.

## Steps to Run
1. Run all cells sequentially.

In [ ]:
import sys, os, pathlib
project_root = pathlib.Path.cwd().parent
sys.path.insert(0, str(project_root))
from src.graph import nxgraph
import src.agent as agent
from utils.plotter import draw_graph, draw_gossip_step_by_step, start_new_run, create_round_gif
#from utils.simulator_old import get_network_stats, reset_simulation
from utils.simulator_new import simulate_round_agent_driven, get_network_stats, reset_simulation
from widgets.graph_widget import display_graph_widgets, get_graph, is_graph_generated
from widgets.init_widget import display_init_widgets, set_graph_data
from widgets.scenario_widget import display_scenario_widgets
from widgets.simulation_widget import display_simulation_widgets

import ipywidgets as widgets
from IPython.display import display, clear_output


Below is the code for topologic comparison between BA, and ER graph

In [ ]:
# refactored code of above, storing data with dictionary
from src.graph import nxgraph
from utils.plotter import draw_graph
from src.analytics import clustering_co, path_length

# Parameters
node = 25
seed = 69
lower_ut = 50
upper_ut = 50
FILE_PIECES = 15
G = nxgraph()

# Unified storage
graph_data = {
    'BA': [],
    'ER': []
}

# Generate BA graphs and store metrics
for m in range(1, 5):
    ba = G.BA_graph(
        nodes=node,
        edges=m,
        seed=seed,
        weighted=True,
        lower_ut=lower_ut,
        upper_ut=upper_ut
    )

    avg_deg = sum(dict(ba.degree()).values()) / ba.number_of_nodes()
    cluster = round(clustering_co(ba), 2)
    path_L = round(path_length(ba), 2)
    edge_count = ba.number_of_edges()
    node_count = ba.number_of_nodes()

    graph_data['BA'].append({
        'graph': ba,
        'avg_degree': avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length': path_L,
        'num_nodes': node_count,
        'num_edges': edge_count
    })

    print(f"\nBA_graph (m={m})")
    draw_graph(ba, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"Average cluster coefficient: {cluster}")
    print(f"Average path length: {path_L}")
    print(f"Number of edges: {edge_count}")

# Generate ER graphs based on BA average degree
for ba_info in graph_data['BA']:
    avg_deg = ba_info['avg_degree']
    edge_count = int(node * avg_deg / 2)

    er = G.ER_Graph_nm(
        nodes=node,
        edges=edge_count,
        weighted=True,
        seed=seed,
        lower_ut=lower_ut,
        upper_ut=upper_ut
    )

    cluster = round(clustering_co(er), 2)
    path_L = round(path_length(er), 2)
    node_count = er.number_of_nodes()

    graph_data['ER'].append({
        'graph': er,
        'avg_degree': avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length': path_L,
        'num_nodes': node_count,
        'num_edges': edge_count
    })

    print(f"\nER_graph (matched to BA avg_deg={round(avg_deg,2)})")
    draw_graph(er, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"Average cluster coefficient: {cluster}")
    print(f"Average path length: {path_L}")
    print(f"Number of edges: {edge_count}")

print(graph_data['BA'])

Below is the code for scalability test

In [ ]:
# refactored code
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Scenario parameters
seed             = 69
edges_m          = 2
lower_ut         = 50
upper_ut         = 50
FILE_PIECES      = 15

n_seeders        = 1
sim_seed         = seed
search_mode      = 'Realistic'
neighbor_selection = 'Random'
ttl              = 5
cleanup_queries  = True
single_agent     = 0
save_images      = False
visualize_output = True
output_area      = widgets.Output()

G = nxgraph()

# Using a dictionary to store generated data
scal_test_graph_data = {
    'BA': [],
    'ER': []
}

# Generating BA graph
for n in range(10, 160, 20):
    ba = G.BA_graph(
        nodes    = n,
        edges    = edges_m,
        seed     = seed,
        weighted = True,
        lower_ut = lower_ut,
        upper_ut = upper_ut
    )

    node_count = ba.number_of_nodes()
    edge_count = ba.number_of_edges()
    avg_deg    = sum(dict(ba.degree()).values()) / node_count
    cluster    = round(clustering_co(ba), 2)
    path_L     = round(path_length(ba), 2)
    max_rounds = int(round(1.25 * n + 35, 0))

    scal_test_graph_data['BA'].append({
        'graph'                 : ba,
        'num_nodes'             : node_count,
        'num_edges'             : edge_count,
        'avg_degree'            : avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length'   : path_L,
        'max_rounds'            : max_rounds,
        'total_queries'         : 0,
        'total_hits'            : 0,
        'total_transfers'       : 0
    })

    print(f"\nBA_graph n={n}, m={edges_m}")
    draw_graph(ba, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"  | avg_deg={avg_deg:.2f}, C={cluster}, L={path_L}, edges={edge_count}")

# Generating ER graph
for ba_rec in scal_test_graph_data['BA']:
    n       = ba_rec['num_nodes']
    avg_deg = ba_rec['avg_degree']
    m_er    = int(n * avg_deg / 2)

    er = G.ER_Graph_nm(
        nodes    = n,
        edges    = m_er,
        weighted = True,
        seed     = seed,
        lower_ut = lower_ut,
        upper_ut = upper_ut
    )

    node_count = er.number_of_nodes()
    edge_count = er.number_of_edges()
    cluster    = round(clustering_co(er), 2)
    path_L     = round(path_length(er), 2)
    max_rounds = ba_rec['max_rounds']  # reuse same TTL logic

    scal_test_graph_data['ER'].append({
        'graph'                 : er,
        'num_nodes'             : node_count,
        'num_edges'             : edge_count,
        'avg_degree'            : avg_deg,
        'clustering_coefficient': cluster,
        'average_path_length'   : path_L,
        'max_rounds'            : max_rounds,
        'total_queries'         : 0,
        'total_hits'            : 0,
        'total_transfers'       : 0
    })

    print(f"\nER_graph n={n}, m≈{m_er}")
    draw_graph(er, edge_labels=None, total_pieces=FILE_PIECES)
    print(f"  | avg_deg={avg_deg:.2f}, C={cluster}, L={path_L}, edges={edge_count}")

# nested for loop for BA and ER
for kind in ('BA', 'ER'):
    for rec in scal_test_graph_data[kind]:
        Gk       = rec['graph']
        rounds   = rec['max_rounds']
        rec_q    = 0
        rec_h    = 0
        rec_t    = 0

        # initialise simulation
        set_graph_data(Gk, FILE_PIECES)
        agent.assign_n_seeders(Gk, n=n_seeders, seed=sim_seed)
        agent.initialize_file_sharing(
            Gk, FILE_PIECES,
            seed              = sim_seed,
            distribution_type = 'n_seeders',
            n_seeders         = n_seeders
        )

        # simulating
        for r in range(1, rounds + 1):
            result = simulate_round_agent_driven(
                Gk,
                FILE_PIECES,
                seed                    = sim_seed,
                cleanup_completed_queries=cleanup_queries,
                search_mode             = search_mode,
                current_round           = r,
                neighbor_selection      = neighbor_selection,
                single_agent            = (single_agent or None)
            )

            queries, hits, _ = result['message_rounds']
            transfers       = result['transfers']

            rec_q += len(queries)
            rec_h += len(hits)
            rec_t += len(transfers)

        rec['total_queries']   = rec_q
        rec['total_hits']      = rec_h
        rec['total_transfers'] = rec_t

        print(f"\n{kind} n={rec['num_nodes']} → Queries = {rec_q}, Hits = {rec_h}, Transfers = {rec_t}")
        draw_gossip_step_by_step(Gk, result['message_rounds'], result['transfers'], 
                            FILE_PIECES, round_num, save_images=False, 
                            max_ttl=ttl, show_debug_info=False)

# evaluation of results
fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharex=True)
for row, kind in enumerate(('BA', 'ER')):
    recs      = scal_test_graph_data[kind]
    nodes     = [r['num_nodes']       for r in recs]
    queries   = [r['total_queries']   for r in recs]
    hits      = [r['total_hits']      for r in recs]
    transfers = [r['total_transfers'] for r in recs]

    axes[row][0].plot(nodes, queries, marker='o')
    axes[row][0].set_title(f"{kind}: Queries vs Nodes")
    axes[row][1].plot(nodes, hits, marker='s', color='green')
    axes[row][1].set_title(f"{kind}: Hits vs Nodes")
    axes[row][2].plot(nodes, transfers, marker='^', color='red')
    axes[row][2].set_title(f"{kind}: Transfers vs Nodes")

for ax in axes.flat:
    ax.set_xlabel("Number of Nodes")
    ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
########################################################
# MAIN CONTROLLER
########################################################
if not is_graph_generated(): raise RuntimeError("Graph must be generated before proceeding. Run the first cell and click 'Generate Graph' to create a network.")
G, FILE_PIECES = get_graph()
set_graph_data(G.graph, FILE_PIECES)

print(f"Using graph with {G.graph.number_of_nodes()} nodes and {G.graph.number_of_edges()} edges")

# Display initialization widgets
display_init_widgets()
display_scenario_widgets()
display_simulation_widgets()


In [7]:
# Launching controller via a hybrid code & UI
# select which generated graph to use in either graph_store_ba, or graph_store_er
chosen_graph = graph_store_ba[0]
set_graph_data(chosen_graph, FILE_PIECES)

print(f"Using graph with {G.graph.number_of_nodes()} nodes and {G.graph.number_of_edges()} edges")

# Display initialization widgets
display_init_widgets()
display_scenario_widgets()
display_simulation_widgets()

Using graph with 25 nodes and 84 edges


Use the controls above to reset the simulation and apply file distributions.
Make sure to generate a graph first using the graph generation widgets.


## PLACEHOLDER ##.
this is output area Output()


Generate a graph and initialize file sharing first.


In [ ]:
print(f"\nFinal graph visualization:")
gifs = create_round_gif(duration=300)
draw_graph(G.graph, total_pieces=FILE_PIECES)